# Notebook Summary

In this notebook I optimize a KNN model

In [1]:
import autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import Classes

In [2]:
import pickle
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import ADASYN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

# Pickle in Data

In [4]:
# Pickle in factorized data

path = r"C:\Users\Andrew\Documents\Metis\TikTok_Hit_Predictor\Pickle\supervised_factorized.pkl"

df = pickle.load(open(path,'rb'))
df.head(2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year,spotify_artists,success
0,0.880,0.501,2.0,-6.774,1.0,0.062,0.0494,0.0695,0.4360,0.459,120.038,2020.0,0,1.0
1,0.935,0.454,1.0,-7.509,1.0,0.375,0.0194,0.0000,0.0824,0.357,133.073,2018.0,1,1.0


# CV KNN

In [5]:
# Seperate features from label

X = df.loc[:,['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','year','spotify_artists']]

y = df['success']

In [6]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [7]:
# Oversample

ada = ADASYN(random_state=42)

# Oversample training data
X_adasyn_tr, y_adasyn_tr = ada.fit_resample(X_train,y_train)

In [8]:
# Scale features 
scaler = StandardScaler()
X_adasyn_tr = scaler.fit_transform(X_adasyn_tr)

In [9]:
# specify "parameter distributions" rather than a "parameter grid"
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']
param_dist = dict(n_neighbors=k_range, weights=weight_options)

In [10]:
# create a parameter grid: map the parameter names to the values that should be searched
param_grid = dict(n_neighbors=k_range, weights=weight_options)
print(param_grid)

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'weights': ['uniform', 'distance']}


In [11]:
# Define KNN
knn = KNeighborsClassifier()

In [12]:
# n_iter controls the number of searches
rand = RandomizedSearchCV(knn, param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=42)
rand.fit(X_adasyn_tr, y_adasyn_tr)
rand.cv_results_

{'mean_fit_time': array([0.02309914, 0.02293763, 0.02289209, 0.02371006, 0.02284214,
        0.02304583, 0.02329388, 0.02289019, 0.02320864, 0.0236172 ]),
 'std_fit_time': array([0.00031483, 0.00084126, 0.00028127, 0.00059262, 0.00053741,
        0.00037344, 0.00099755, 0.00069059, 0.00039231, 0.00064798]),
 'mean_score_time': array([0.09803848, 0.11643262, 0.20433989, 0.19791222, 0.14766216,
        0.21684854, 0.1823195 , 0.21411512, 0.16821129, 0.20579994]),
 'std_score_time': array([0.00370722, 0.00473737, 0.00239599, 0.00415684, 0.00375835,
        0.00200153, 0.0059155 , 0.00249291, 0.00263487, 0.00294983]),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
                    'distance', 'uniform', 'distance', 'uniform',
                    'uniform', 'distance'],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_n_neighbors': 

In [14]:
# examine the best model
print(rand.best_score_)
print(rand.best_params_)
print(rand.best_estimator_)

0.9236213235294117
{'weights': 'uniform', 'n_neighbors': 1}
KNeighborsClassifier(n_neighbors=1)


# Applying best params

In [15]:
# Seperate features from label

X = df.loc[:,['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','year','spotify_artists']]

y = df['success']

In [16]:
#Split data into 3: 60% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [17]:
# Oversample

ada = ADASYN(random_state=42)

# Oversample training data
X_adasyn_tr, y_adasyn_tr = ada.fit_resample(X_train,y_train)
X_adasyn_test, y_adasyn_test = ada.fit_resample(X_test,y_test)

In [18]:
# Scale features 
scaler = StandardScaler()
X_adasyn_tr = scaler.fit_transform(X_adasyn_tr)
X_adasyn_test = scaler.fit_transform(X_adasyn_test)

In [48]:
# run initial logistic regression
# Define KNN
model = KNeighborsClassifier(n_neighbors = 10, weights = 'uniform')
model.fit(X_adasyn_tr, y_adasyn_tr)
y_predict = model.predict(X_adasyn_test)

In [49]:
Classes.train_scores(model,X_adasyn_tr, y_adasyn_tr)

Train Scores
Accuracy score:  91.45%


In [50]:
Classes.test_scores(y_adasyn_test,y_predict)

Test Scores
Score:  87.40%
F1 score:  88.21%
Precision:  82.68%,  Test Recall:  94.54%


10 neighbors appears to be the best K valuem, the grid search i tried was too wide

# Pickle out model

In [52]:
# Pickle model
path = r"C:\Users\Andrew\Documents\Metis\TikTok_Hit_Predictor\Pickle\knn_model.pkl"
pickle.dump(model, open(path, 'wb'))